In [59]:
from random import randint
import json
import random
from IPython.display import clear_output
#start of every round, initialize the player
#self.health is max health and currHealth is current health 
#elemental resistances is a dictionary with elements and their respective resistances 
class Player:
    def __init__(self, energy, health, currHealth, gold, experience, level, weapon, armor, skills, elementalResistances):
        self.energy = energy
        self.health = health + armor.health
        self.currHealth = currHealth
        self.gold = gold
        self.weapon = weapon
        self.armor = armor
        self.experience = experience
        self.level = level
        self.turnGauge = 0
        self.skillList = skills 
        self.criticalRate = 0.05 + weapon.criticalRate
        self.speed = 7 + weapon.speed + armor.speed
        self.attackDamage = 5 + weapon.attackDamage
        self.magicDamage = 25 + weapon.magicDamage
        self.physicalDefense = 10 + armor.physicalDefense
        self.magicDefense = 10 + armor.magicDefense
        self.luck = 0.05 + armor.luck
        #elemental resistance is mostly based on the armor 
        self.elementalResistances = {'Fire':0+armor.elementalResistances['Fire'], 
                                     'Water':0+armor.elementalResistances['Water'], 
                                     'Electric':0+armor.elementalResistances['Electric'],
                                     'Ice':0+armor.elementalResistances['Ice'],
                                     'Dark':0+armor.elementalResistances['Dark'],
                                     'Light':0+armor.elementalResistances['Light'],
                                     'Physical':0+armor.elementalResistances['Physical']
                                    }
    
    def adjustHealth(self, health): #Health adjustments can be positive or negative 
        self.currHealth += health
    
    def adjustNegativeEnergy(self, energy):
        self.energy -= energy
        
    def adjustGold(self, gold): #gold adjustments can be positive or negative
        self.gold += gold
    
    def changeWeapon(self, weapon): 
        self.weapon = weapon
    
    def resetTurnGauge(self):
        self.turnGauge = 0
    
    def changeTurnGauge(self, value):
        self.turnGauge += value
    
    def getTurnGauge(self):
        return self.turnGauge
    
    def changeArmor(self, armor):
        self.armor = armor
    
    def adjustattackDamage(self, value):
        self.attackDamage += value

    def adjustmagicDamage(self, value):
        self.magicDamage += value
    
    def adjustExperience(self, value):
        self.experience += value
        
    def adjustLevel(self, value):
        self.level += value
        
    def setExperience (self, value):
        self.experience = value
    
    def printElementalResistances(self):
        for y in self.elementalResistances[x]:
            print (y,':',self.elementalResistances[x][y])
    
    def getWeapon(self):
        return self.weapon
    
    def getArmor(self):
        return self.armor
    
    def getSpeed(self):
        return self.speed
    
    def getCurrHealth(self):
        return self.currHealth
    
    def getEnergy(self):
        return self.energy
    
    def getAttackDamage(self):
        return self.attackDamage

    def getMagicDamage(self):
        return self.magicDamage

    def getPhysicalDefense(self):
        return self.physicalDefense

    def getMagicalDefense(self):
        return self.magicDefense
    
    def getExperience(self):
        return self.experience
    
    def getLevel(self):
        return self.level
    
class Weapon:
    def __init__(self, name, speed, attackDamage, magicDamage, criticalRate, element, scope):
        self.name = name
        self.speed = speed
        self.attackDamage = attackDamage 
        self.magicDamage = magicDamage 
        self.criticalRate = criticalRate
        self.element = element
        self.scope = scope
        
    def getName(self):
        return self.name
    
    def getElement(self):
        return self.element

    def getScope(self):
        return self.scope

class Armor:
    def __init__(self, name, speed, health, physicalDefense, magicDefense, luck, elementalResistances):
        self.name = name
        self.speed = speed
        self.physicalDefense = physicalDefense
        self.health = health
        self.magicDefense = magicDefense 
        self.luck = luck 
        self.elementalResistances = elementalResistances

    def getName(self):
        return self.name

class Enemy:
    def __init__(self, health, skillDictionary, dropsDictionary, name, goldReward, experienceReward, speed, attackDamage, magicDamage, 
                 physicalDefense, magicDefense, elementalResistances):
        self.health = health
        self.skillDictionary = skillDictionary
        self.dropsDictionary = dropsDictionary
        self.name = name
        self.energy = 100
        self.goldReward = goldReward
        self.experienceReward = experienceReward
        self.speed = speed
        self.turnGauge = 0
        self.attackDamage = attackDamage
        self.magicDamage = magicDamage
        self.physicalDefense = physicalDefense
        self.magicDefense = magicDefense 
        self.elementalResistances = elementalResistances
    
    def adjustHealth(self, health):
        self.health += health
    
    #we're actually not ever checking the monsters energy,
    #this is just to make things smooth when making the functions
    #that execute skills.
    def adjustNegativeEnergy(self, energy):
        self.energy -= energy
    
    def getName(self):
        return self.name
    
    def modifyName(self, numberName):
        self.name = self.name + str(numberName)
    
    def getHealth(self):
        return self.health
    
    def getGoldReward(self):
        return self.goldReward
    
    def getExperienceReward(self):
        return self.experienceReward
    
    def getSpeed(self):
        return self.speed

    def resetTurnGauge(self):
        self.turnGauge = 0
    
    def changeTurnGauge(self, value):
        self.turnGauge += value
    
    def getTurnGauge(self):
        return self.turnGauge
    
    #given the self.skillDictionary that has each
    #skill and their percent chance of being used,
    #pick a random skill from that dictionary. 
    def randomSkillSelection(self):
        skillSelection = random.randint(0, 100)
        for skill in self.skillDictionary:
            #self.skillDictionary.get(skill) returns the percent chance that the 
            #enemy will use that specific skill. 
            #For this to work, skillDictionary MUST be in order from least to greatest percentage.
            if (skillSelection <= self.skillDictionary.get(skill)):
                return skill

class Skill:
    def __init__(self, name, baseDmg, variationDmg, scope, energyCost, element, skillType):
        #scope: determines if the target is one or multiple enemies
        #skillType: determines if the Skill is magical or physical
        self.name = name
        self.baseDmg = baseDmg
        self.variationDmg = variationDmg 
        self.scope = scope
        self.energyCost = energyCost
        self.element = element 
        self.skillType = skillType
    
    def getEnergy(self):
        return self.energyCost
    
    def getName(self):
        return self.name
    
    def getScope(self):
        return self.scope
    
    def getElement(self):
        return self.element
    
    def adjustScope(self, scope):
        self.scope = scope
        
    def adjustName(self, name):
        self.name = name
        
    def adjustElement(self, element):
        self.element = element
    
    def adjustSkillType(self, skilltype):
        self.skillType = skilltype
    
    def executeSkill(self, target, user): #skill type refers to physical, magical, or neutral damage 
        rawDamage = randint(self.baseDmg-self.variationDmg, self.baseDmg+self.variationDmg) #establish lower and upper bounds of damage 
        if (self.skillType == 'Physical'):
            user.adjustNegativeEnergy(self.energyCost)
            overallDamage = rawDamage + user.attackDamage - target.physicalDefense - target.elementalResistances[self.element]
            #below conditional statement is used to calculate if the hit is a critical strike
            if (random.uniform(0, 1) <= user.criticalRate):
                overallDamage *= 1.5
                print("Critical Strike!")
            #the two below conditional statements are used to calculate elemental resistances 
            if (target.elementalResistances[self.element] > 5):
                print("Target is resistant to this element!")
            if (target.elementalResistances[self.element] < 0):
                print("Target is weak to this element!")
            #below conditional statement is to make sure the damage does not go negative before the next line 
            if (overallDamage < 0):
                overallDamage = 0 
            target.adjustHealth(-1 * overallDamage)
            #the two below conditional statements print a statement based on if the user of the skill was the player or the enemy
            if (isinstance(user, Player)): 
                print("You dealt " + str(overallDamage) + " damage to the enemy " + target.name + "!" )
                print(target.getName() + " now has " + str(target.getHealth()) + " health!\n")
                #time.sleep in order to give the user time to read the output 
                time.sleep(2)
            if (isinstance(user, Enemy)):
                print( user.getName() + " dealt " + str(overallDamage) + " damage to you!" )
                print("You now have " + str(target.getCurrHealth()) + " health!\n")
                time.sleep(2)
            
        #same process as above, but taking into account magic damage and removing the ability to critical hit    
        if (self.skillType == 'Magical'):
            user.adjustNegativeEnergy(self.energyCost)
            overallDamage = rawDamage + user.magicDamage - target.magicDefense - target.elementalResistances[self.element]
            if (target.elementalResistances[self.element] > 5):
                print("Target is resistant to this element!")
            if (target.elementalResistances[self.element] < 0):
                print("Target is weak to this element!")
            if (overallDamage < 0):
                overallDamage = 0 
            target.adjustHealth(-1 * overallDamage)
            if (isinstance(user, Player)):
                print("You dealt " + str(overallDamage) + " damage to the enemy " + target.name + "!" )
                print(target.getName() + " now has " + str(target.getHealth()) + " health!\n")
                time.sleep(2)
            if (isinstance(user, Enemy)):
                print( user.getName() + " used " + self.name + " and dealt " + str(overallDamage) + " damage to you!" )
                print("You now have " + str(target.getCurrHealth()) + " health!\n")
                time.sleep(2)
            ##put a function here that checks the health for game overs and/or enemy defeats 
            #or just check after we call this function 
        #clear_output()
        return overallDamage 
    
class BuffDebuffSkill:
    def __init__(self, name, energy, stat, adjustment, skillType):
        self.name = name
        self.energyCost = energy
        self.stat = stat #tracks which stat in particular will be adjusted 
        self.skillType = skillType #tracks if the stat is a buff or debuff
        if (self.skillType == "Buff"):
            self.adjustment = adjustment 
        else:
            self.adjustment = adjustment * -1 
    
    def executeSkill(self, target, user):
        if (self.stat == "Attack Damage"):
            target.adjustattackDamage(self.adjustment)
        if (self.stat == "Magic Damage"):
            target.adjustmagicDamage(self.adjustment)  

    def getName(self):
        return self.name
        
    def getEnergy(self):
        return self.energyCost

    def getType(self):
        return self.skillType
    
    def getStat(self):
        return self.stat


In [60]:
#Below are the initializations for the skills, weapons, and armor. 
# Skill(Name, base dmg, var dmg, scope, energy cost, element, skillType)
fireballSpell = Skill("Fireball", 7.5, 2.5, "Single", 8, 'Fire', "Magical")
darkStrike = Skill("Dark Strike", 7.5, 2.5, "Single", 8, 'Dark', "Magical")
freezeSpell = Skill("Freezing Bolt", 7.5, 2.5, "Single", 8, 'Ice', "Magical")
motivate = BuffDebuffSkill("Motivate", 5, "Attack Damage", 5, "Buff")
zap = Skill("Zap", 7.5, 2.5, "Single", 8, "Electric", "Magical")
basicAttack = Skill("Attack", 3, 1, "Single", 0, "Fire", "Physical")

# Weapon (Name, Speed, Attack, Magic, Crit Rate, Element)
# Armor (Name, Speed, Health, physical defense, magical defense, luck, elemental resist)
scythe = Weapon("Flaming Scythe", 0, 3, 3, 0.05, 'Fire', "Multiple")
silverSword = Weapon("Silver Sword", 5, 1, 0, 0.03, 'Physical', "Single")
plating = Armor("Plating", 2, 15, 1, 1, 1, {'Fire':3, 'Water': 2, 'Electric': 0, 'Ice': 5, 'Dark': 3, 'Light': -1, 'Physical': 0})

In [61]:
######################################################################
#
# All code in this section is for the preparation screen, where 
# the player can equip skills, armor, weapons, etc.
#
######################################################################
import json
import time
from IPython.display import clear_output

    
#let's say they pass in 'Fireball', we want it to return fireballSpell. 
#We could have a list of skills and we check each name to see if it 
#matches the parameter! 

#The user enters in the string representation, the program
#reads from the JSON that has the skill String Representations,
#and then it should blurt out the Skill version of the string representation. 

#this list should have every attainable spell and attack 
everySkill = [basicAttack, fireballSpell, freezeSpell, motivate, zap] 
everyWeapon = [scythe, silverSword] 
everyArmor = [plating] 


#this function loops through the
#everySkill list and appends the skill to the
#given list if the skill is found in everySkill
def findSkill (specificSkillName, skillList):
    for skill in everySkill:
        if (skill.getName() == specificSkillName):
            skillList.append(skill)

def findWeapon (specificWeaponName, weaponList):
    for weapon in everyWeapon:
        if (weapon.getName() == specificWeaponName):
            weaponList.append(weapon)            

def findArmor (specificArmorName, armorList):
    for armor in everyArmor:
        if (armor.getName() == specificArmorName):
            armorList.append(armor)  
            

def skillEquality (skillName, skill2):
    return skillName == skill2.getName()
        
def printList(list):
    for item in list:
        print (item)

def printSkills(list):
    for item in list:
        if (isinstance(item, BuffDebuffSkill)):
            print(item.getName() + " (Energy Cost: " + str(item.getEnergy()*-1) + ", Stat: " + item.getStat() + ", Type: " + item.getType() + ")"  )
        else: 
            print(item.getName() + " (Energy Cost: " + str(item.getEnergy()*-1) + ", Element: " + item.getElement() + ", Scope: " + item.getScope() + ")"  )

#read through the json file and keep track
#of all the skills that are unlocked. 
with open('unlockables.json') as json_file:
    data = json.load(json_file)
#read in the unlockables and store them in this list that 
#can be accessed everywhere
allUnlocksFromJson = data
unlockedSkills_string = allUnlocksFromJson["skillUnlocks"]
unlockedWeapons_string = allUnlocksFromJson["weaponUnlocks"]
unlockedArmors_string = allUnlocksFromJson["armorUnlocks"]

#THIS SHOULD BE MOVED TO START OF PROGRAM AFTER
with open('saveData.json') as json_file:
    data = json.load(json_file)
allSaveData = data
#the next few lines equip the skills that were listed as 
#equipped by the save file. 
equippedSkills_string = allSaveData["equippedSkills"]
equippedSkills = [] 
for eachSkill in equippedSkills_string:
    #for every string representation of a skill in the equippedSkills_string list,
    #find the corresponding skill from the everySkill list and add it to equippedSkills
    findSkill(eachSkill, equippedSkills)
#equips the current weapon from the save file 
for weapon in everyWeapon:
    if (weapon.getName().upper() == allSaveData["currentWeaponEquip"].upper()):
        currentWeapon = weapon
#equips the current armor from the save file 
for armor in everyArmor:
    if (armor.getName().upper() == allSaveData["currentArmorEquip"].upper()):
        currentArmor = armor
#synchronize the inventories
weaponInventory = []
armorInventory = []
for weapon in allSaveData["weaponInventory"]:
    findWeapon(weapon, weaponInventory)
for armor in allSaveData["armorInventory"]:
    findArmor(armor, armorInventory)


    
    
    
# player parameters: (energy, health, currHealth, gold, experience, level, weapon, armor, skills, elementalResistances):    
protagonist = Player(allSaveData["energy"], allSaveData["health"], allSaveData["currHealth"], allSaveData["gold"], allSaveData["experience"], allSaveData["level"], currentWeapon, currentArmor, equippedSkills, {'Fire':3, 'Water': 2, 'Electric': 0, 'Ice': 5, 'Dark': 3, 'Light': -1, 'Physical': 0})
    
    
    
    
    
#the code below is used for the preparation screen 
#in which the user can choose which armor, weapons, and skills to equip. 
keepEquipping = True
while (keepEquipping):
    firstEquipPrompt = input("What would you like to equip?\nType in 'weapons', 'armor', 'skills' to equip those respectively.\nYou may also type in 'cancel' to stop equipping.")
    if (firstEquipPrompt.upper() == "CANCEL"):
        keepEquipping = False
    
    if (firstEquipPrompt.upper() == "ARMOR"):
        #
        # CODE BELOW FOR EQUIPPING ARMOR
        #
        clear_output()
        keepArmoring = True
        while (keepArmoring):
            invalidEquipArmor = False
            print("You currently have " + protagonist.getArmor().getName() + " equipped.")
            armorInput = input("Which armor would you like to equip?\nType in 'inventory' to see all the armor you have.\nType in 'cancel' to stop equipping weapons.")
            
            if (armorInput.upper() == "INVENTORY"):
                clear_output()
                print("Here are the armor in your inventory:")
                for armor in armorInventory:
                    print (armor.getName())
                    invalidEquipArmor = True
                    
            if (armorInput.upper() == "CANCEL"):
                clear_output()
                keepArmoring = False
                invalidEquipArmor = True
                
            if (protagonist.getArmor().getName().upper() == armorInput.upper()):
                clear_output()
                print("You already have that armor equipped.\n")
                invalidEquipArmor = True
                
            if (invalidEquipArmor == False):
                for armor in armorInventory:
                    if (armor.getName().upper() == armorInput.upper()):
                        protagonist.changeArmor(armor)
                        clear_output()
                        print("Successfully equipped " + protagonist.getArmor().getName() + "!")
                        keepArmoring = False
                if (keepArmoring):
                    clear_output()
                    print("Invalid selection.\n")

        with open('saveData.json', 'w') as outfile:
            allSaveData["currentArmorEquip"] = protagonist.getArmor().getName()
            json.dump(allSaveData, outfile) 
        #
        # CODE FOR EQUIPPING ARMOR ENDS HERE
        #            
    
    if (firstEquipPrompt.upper() == "WEAPONS"):
        #
        # CODE BELOW FOR EQUIPPING WEAPONS
        #
        clear_output()
        keepWeaponing = True 
        while (keepWeaponing):
            invalidEquipWeapon = False
            print("You currently have " + protagonist.getWeapon().getName()  + " equipped.")
            weaponInput = input("Which weapon would you like to equip?\nType in 'inventory' to see all the weapons you have.\nType in 'cancel' to stop equipping weapons.")
            
            if (weaponInput.upper() == "INVENTORY"):
                clear_output()
                print("Here are the weapons in your inventory:")
                for weapon in weaponInventory:
                    print (weapon.getName())
                    invalidEquipWeapon = True
                    
            if (weaponInput.upper() == "CANCEL"):
                clear_output()
                keepWeaponing = False
                invalidEquipWeapon = True
                
            if (protagonist.getWeapon().getName().upper() == weaponInput.upper()):
                clear_output()
                print("You already have that weapon equipped.\n")
                invalidEquipWeapon = True
                
            if (invalidEquipWeapon == False):
                for weapon in weaponInventory:
                    if (weapon.getName().upper() == weaponInput.upper()):
                        protagonist.changeWeapon(weapon)
                        clear_output()
                        print("Successfully equipped " + protagonist.getWeapon().getName() + "!")
                        keepWeaponing = False
                if (keepWeaponing):
                    clear_output()
                    print("Invalid selection.\n")

        with open('saveData.json', 'w') as outfile:
            allSaveData["currentWeaponEquip"] = protagonist.getWeapon().getName()
            json.dump(allSaveData, outfile) 
        #
        # CODE FOR EQUIPPING WEAPONS ENDS HERE
        #
    if (firstEquipPrompt.upper() == "SKILLS"):
        #
        # CODE BELOW FOR EQUIPPING SKILLS 
        #
        clear_output()
        keepPrompting = True
        while(keepPrompting): #keep asking the user to equip a skill until its valid
            #otherOptions will track if the user picks one of the other options (i.e if they choose to view equips or unlocks instead)
            #checkInvalidSkill will check if the user enters in a valid skill input
            #continueEquipping will be used later to track if the user can keep equipping skills 
            otherOptions = False
            checkInvalidSkill = True
            continueEquipping = True
            userInput = input("Which skill would you like to equip?\nType in 'equips' to see what you currently have equipped and 'unlocks' to see all the skills you have unlocked.\nYou can also type in 'cancel' to stop equipping skills.")
            clear_output()
            if (userInput.upper() == "EQUIPS"):
                print("Here are the skills you currently have equipped:\n")
                printList(equippedSkills_string)
                otherOptions = True
            if (userInput.upper() == "UNLOCKS"):
                print("Here are the skills you currently have unlocked:\n")
                for item in unlockedSkills_string:
                    printEquipped = False
                    for skill in equippedSkills_string:
                        if (item == skill):
                            printEquipped = True
                    if (printEquipped):
                        print (item + " (Equipped)")
                    else:
                        print (item)
                otherOptions = True
            if (userInput.upper() == "CANCEL"):
                keepPrompting = False   
                checkInvalidSkill = False
            if (otherOptions == False):
                #this for loop checks if the skill input the user submitted is an unlocked skill
                for skill_string in unlockedSkills_string:
                        if (userInput.upper() == skill_string.upper()):
                            #this for loop makes sure the skill the user inputted is not CURRENTLY equippedt
                            for skillsEquipped in equippedSkills_string:
                                if (userInput.upper() == skillsEquipped.upper()):
                                    print("You already have " + userInput + " equipped!")
                                    continueEquipping = False
                            #If the skill is not already equipped, then proceed with the equip process
                            if (continueEquipping):
                                clear_output()
                                print("Here are the skills you currently have equipped:\n")
                                #print every skill you have equipped currently 
                                printList(equippedSkills_string)
                                replaceInput = input("Which skill would you like to replace with " + skill_string + "?\nYou can also type in 'none' to cancel this selection.\n")
                                if (replaceInput.upper() == "ATTACK"):
                                    clear_output()
                                    checkInvalidSkill = False
                                    print("You can't replace your default attack.")
                                    time.sleep(3)
                                    clear_output()
                                elif (replaceInput.upper() == 'NONE'):
                                    checkInvalidSkill = False
                                    clear_output()
                                else:
                                    #replace the skill at the index where we find it 
                                    for index, skill3 in enumerate(equippedSkills_string):
                                        if (skill3.upper() == replaceInput.upper()):
                                            equippedSkills_string[index] = skill_string
                                            clear_output()
                                            print(userInput + " was successfully equipped!")
                                            checkInvalidSkill = False   
                                            continueEquipping = input("Would you like to continue equipping skills? Type in 'yes' to continue equipping skills.")
                                            if (continueEquipping.upper() == "YES"):
                                                keepPrompting = True
                                                clear_output()
                                            else:
                                                keepPrompting = False
                                        
                if (checkInvalidSkill):
                    print(userInput + " is not a valid selection.")

        #equippedSkills is the same as equippedSkills_string,
        #except the strings are turned into their Skill representations 

        for eachSkill in equippedSkills_string:
            #for every string representation of a skill in the equippedSkills_string list,
            #find the corresponding skill from the everySkill list and add it to equippedSkills
            findSkill(eachSkill, equippedSkills)
        with open('saveData.json', 'w') as outfile:
            allSaveData["equippedSkills"] = equippedSkills_string
            json.dump(allSaveData, outfile) 
        #
        # Code for equipping skills ends here 
        #
######################################################################
#
# Code for the preparation screen ends here 
# NOTE: The dumping into the JSON could and should be moved off the if statements
######################################################################

#should track progress on a stage by stage basis
#and save data after every stage
#maybe include a retry option at the end of every stage
#that just loads the save data 

What would you like to equip?
Type in 'weapons', 'armor', 'skills' to equip those respectively.
You may also type in 'cancel' to stop equipping.cancel


In [55]:
allUnlocksFromJson

{'skillUnlocks': ['Attack',
  'Fireball',
  'Freezing Bolt',
  'Motivate',
  'Zap',
  'Dark Strike'],
 'weaponUnlocks': ['Scythe', 'Silver Sword'],
 'armorUnlocks': ['Plating']}

In [7]:
allUnlocksFromJson
armorInventory
weaponInventory

[<__main__.Weapon at 0x13ddd8f4b80>, <__main__.Weapon at 0x13ddd8f4970>]

In [66]:
import json
#unlockables = {}
#unlockables['skillUnlocks'] = [] ##so one entry in the unlockables dictionary is a list of skills that are unlocked 
#unlockables['skillUnlocks'].append('Fireball')
#unlockables['skillUnlocks'].append('Freezing Bolt')
#unlockables['skillUnlocks'].append('Motivate')
#unlockables['skillUnlocks'].append('Electric')

#this would probably be at the end of the program
#where we dump all the progress into the JSON file. 

#so the previous cell would take in the unlockables from the JSON
#and this line below would send it back into the JSON after we've updated
#it with new unlockables
with open('unlockables.json', 'w') as outfile:
    json.dump(allUnlocksFromJson, outfile)

#once we develop the program more,
#we can just have a custom starting unlockables.json
#that the player starts with.

#for now, this cell should create/modify the unlockables

#at the end of the program, the json file should probably be modified
#with any new unlocks. 

In [62]:
def levelUpCalculation(theProtagonist, experiencePointsGained, saveFile):
    #every level requires 25 more exp than the previous 
    levelCap = theProtagonist.getLevel() * 25
    theProtagonist.adjustExperience(experiencePointsGained)
    if (theProtagonist.getExperience() >= levelCap):
        #leftOverExperience after leveling up 
        leftoverExperience = theProtagonist.getExperience() - levelCap
        #reset experience after leveling up 
        theProtagonist.setExperience(0)
        #add over the leftover experience 
        theProtagonist.adjustExperience(leftoverExperience)
        #actually level up
        theProtagonist.adjustLevel(1)
        print("You gained " + str(experiencePointsGained) + " experience and leveled up to level " + str(theProtagonist.getLevel()) + "!")
        #bottom condtional statement just in case the leftOverExperience ended up resulting in another level up. 
        if (theProtagonist.getExperience() >= theProtagonist.getLevel() * 25):
            levelUpCalculation(theProtagonist, 0)
    else:
        print("You gained " + str(experiencePointsGained) + " experience and you are still level " + str(theProtagonist.getLevel()) + ".")
        
    with open(saveFile, 'w') as outfile:
        allSaveData["level"] = theProtagonist.getLevel()
        allSaveData["experience"] = theProtagonist.getExperience()
        json.dump(allSaveData, outfile) 
        
        #add algorithm for adjusting stats based on users level 

In [68]:
import random
import copy
import time
elementResistance = {'Fire':3, 'Water': 2, 'Electric': 0, 'Ice': 10, 'Dark': -10, 'Light': -1, 'Physical': 0}
elementResistance2 = {'Fire':7, 'Water': 2, 'Electric': 0, 'Ice': -3, 'Dark': 10, 'Light': -1, 'Physical': 0}
#initialize slime enemy 
#enemy parameters: (health, skillDictionary, dropsDictionary, name, goldReward, experienceReward, speed,
#attackDamage, magicDamage, 
#physicalDefense, magicDefense, elementalResistances):
slimeEnemy = Enemy(50, {freezeSpell: 75, zap: 100}, {plating: 0.25}, 'Slime', 10, 15, 5, 20, 20, 25, 9, elementResistance)
batEnemy = Enemy(25, {darkStrike: 75, fireballSpell: 100}, {plating: 0.25}, 'Bat', 10, 15, 15, 15, 15, 3, 3, elementResistance2)
monsterList = [slimeEnemy, batEnemy]
basicAttack.adjustElement(protagonist.getWeapon().getElement())
basicAttack.adjustScope(protagonist.getWeapon().getScope())
#given a list of possible monsters, assemble
#a battle troop that the player will face.

def assembleBattleTroop(monsterList, count):
    battleTroopDictionary = {}
    #Create a dictionary based on the monsterList.
    #The keys will be the enemies from the monsterList,
    #and the values will end up being how many dupes of
    #that enemy are in each troop. 
    battleTroopDictionary = dict.fromkeys(monsterList, 1)
    finalBattleTroop = []
    while (count > 0):
        randomEnemyIndex = randint(0, len(monsterList)-1)
        randomEnemy = monsterList[randomEnemyIndex]
        for enemy in finalBattleTroop:
            #check if the enemy being added is already in the troop.
            #if so, increase relevant value in dictionary
            if (enemy.getName() == randomEnemy.getName()):
                battleTroopDictionary[enemy] += 1
        if (battleTroopDictionary[randomEnemy] > 1):
            #deepcopy basically makes the dupe seperate from the original!
            dupe = copy.deepcopy(randomEnemy)
            dupe.modifyName(battleTroopDictionary[randomEnemy])
            finalBattleTroop.append(dupe)
        else: 
            finalBattleTroop.append(randomEnemy)
        count-=1
    return finalBattleTroop


#finally got it working!
#ultimately chose a dictionary since it can
#track repeats for each individual enemy. 

#this function is for when the player is using a skill 
def userSkillBattle(enemyTroops): #enemy troops should be a list of the enemies you're fighting
    #make sure each enemy has a number just in case of dupes
    #while adding, check for dupes 
    #clear_output()
    invalidSkill = True
    invalidEnemy = True
    while(invalidSkill & invalidEnemy):
        print("Your Health: " + str(protagonist.getCurrHealth()) + " | Your Energy: " + str(protagonist.getEnergy()) + " | Your Level: " + str(protagonist.getLevel())) 
        print("Here are the current enemies you are fighting:\n")
        for enemy in enemyTroops:
            print(enemy.getName() + " with " + str(enemy.getHealth()) + " health.")        
        print("\nHere are the skills you currently have equipped:\n")
        #print(basicAttack.getName() + " (Energy Cost: " + str(basicAttack.getEnergy()*-1) + ")"  )
        printSkills(equippedSkills)
        chooseSkill = input("Which skill would you like to use?\n")
        if (chooseSkill.upper() == "SURRENDER"):
            invalidSkill = False
            invalidEnemy = False
        for eachSkill in equippedSkills:
            if (chooseSkill.upper() == eachSkill.getName().upper()):
                invalidSkill = False
                #check if it's a buff/debuff skill and if so,
                #change it up.
                if (eachSkill.getScope() == "Single"):
                    if (protagonist.getEnergy() < eachSkill.getEnergy()):
                        print("You do not have enough energy to use this skill!")
                    else:
                        clear_output()
                        print("Here are the current enemies you are fighting:\n")
                        for enemy in enemyTroops:
                            print(enemy.getName() + " with " + str(enemy.getHealth()) + " health.")
                        target = input("Which enemy would you like to target with " + eachSkill.getName() + "? Type 'back' to select skills again.")
                        clear_output()
                        if (target.upper() == "BACK"):
                            break
                        for enemy in enemyTroops:
                            if (target.upper() == enemy.getName().upper()):
                                eachSkill.executeSkill(enemy, protagonist)
                                if (enemy.getHealth() <= 0):
                                    print("You defeated " + enemy.getName() + "!")
                                    enemyTroops.remove(enemy)
                                invalidEnemy = False
                        if (invalidEnemy == True):
                            clear_output()
                            print("Not a valid target.")
                elif (eachSkill.getScope() == "Multiple"):
                    clear_output()
                    for enemy in enemyTroops:
                        eachSkill.executeSkill(enemy, protagonist)
                        if (enemy.getHealth() <= 0):
                            print("You defeated " + enemy.getName() + "!")
                            enemyTroops.remove(enemy)
        if (invalidSkill):
            print(chooseSkill + " is not an unlocked skill you can use.")
            time.sleep(2)
            clear_output()
            time.sleep(0.5)


def enemyAction(specificEnemy):
    chosenSkill = specificEnemy.randomSkillSelection()
    chosenSkill.executeSkill(protagonist, specificEnemy)
    if (protagonist.getCurrHealth() <= 0):
        print("You lost.")
        exit()

#call battleTime whenever you start a battle
#TODO: Implement drops from the enemy as well as updating the save file based on the users
#health and inventory. 
def battleTime(monsters):
    #since stats will be changing throughout the way with buffs 
    #and debuffs. 
    totalGoldReward = 0
    totalExperienceReward = 0
    initialAttack = protagonist.getAttackDamage()
    initialMagic = protagonist.getMagicDamage()
    initialPhysdef = protagonist.getPhysicalDefense()
    initialMagicdef = protagonist.getMagicalDefense()
    #can add a check to see if buffed version of stats
    #is a percentage better than the initial stats 
    #to see if the buff should be restricted.
    battleTroopers = assembleBattleTroop(monsters, 2)
    #below is for calculating rewards
    tempBattleTroopers = assembleBattleTroop(monsters, 2)
    protagSpeed = protagonist.getSpeed()
    protagonist.resetTurnGauge()
    while(len(battleTroopers) > 0):
        #the turn gauge is adjusted based on the users speed.
        #The more speed the user has, the faster the turn gauge fills up,
        #which means the user can take more actions. 
        protagonist.changeTurnGauge(protagSpeed)
        if (protagonist.getTurnGauge() >= 50):
            userSkillBattle(battleTroopers)
            protagonist.resetTurnGauge()
            clear_output()
        for enemy in battleTroopers:
            enemy.changeTurnGauge(enemy.getSpeed())
            if (enemy.getTurnGauge() >= 50):
                enemyAction(enemy)
                enemy.resetTurnGauge()
                clear_output()
    #if protagonist wins 
    print("You defeated every enemy!")
    time.sleep(0.5)
    for eachMonster in tempBattleTroopers:
        totalGoldReward += eachMonster.getGoldReward()
        totalExperienceReward += eachMonster.getExperienceReward()
    print("Gold gained: " + str(totalGoldReward))
    time.sleep(0.5)
    levelUpCalculation(protagonist, totalExperienceReward, 'saveData.json')
    time.sleep(0.5)
    with open('saveData.json', 'w') as outfile:
        allSaveData["currHealth"] = protagonist.getCurrHealth()
        allSaveData["energy"] = protagonist.getEnergy()
        json.dump(allSaveData, outfile) 

battleTime(monsterList)

You defeated every enemy!
Gold gained: 20
You gained 30 experience and leveled up to level 3!


NameError: name 'theProtagonist' is not defined

In [45]:
monsters = [slimeEnemy]
battleTroopers = assembleBattleTroop(monsters, 2)
totalGoldReward = 0
for eachMonster in battleTroopers:
    totalGoldReward += eachMonster.getGoldReward()
    
print(totalGoldReward)

20
